In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import os
import struct
import hashlib
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
import tensorflow_hub as hub
import tokenization
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from rouge_score import rouge_scorer
from sklearn.cluster import KMeans

In [2]:
def hashhex(s):
    h = hashlib.sha1()
    h.update(s)
    return h.hexdigest()

In [3]:
def get_url_hashes(url_list):
    return [hashhex(url) for url in url_list]

def read_text_file(text_file):
    lines = []
    with open(text_file, "r") as f:
        for line in f:
            lines.append(line.strip().encode('utf-8'))
    return lines

url_file_train = 'CNN-DM/all_train.txt'
url_list_train = read_text_file(url_file_train)
url_hashes_train = get_url_hashes(url_list_train)
print(len(url_hashes_train))

url_file_val = 'CNN-DM/all_val.txt'
url_list_val = read_text_file(url_file_val)
url_hashes_val = get_url_hashes(url_list_val)
print(len(url_hashes_val))

url_file_test = 'CNN-DM/all_test.txt'
url_list_test = read_text_file(url_file_test)
url_hashes_test = get_url_hashes(url_list_test)
print(len(url_hashes_test))

287227
13368
11490


In [36]:
x = url_hashes_train[0]
l = open(os.path.join("CNN-DM","raw","cnn","stories",x+".story")).readlines()

for line in l:
    print(line,end='')

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill.

An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.

MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court.

Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to follow 

In [5]:
def clean(x):
    text = []
    summary = []
    flag = 0
    for line in x:
        if line == '\n':
            continue
        if '@highlight' in line:
            flag = 1
            continue
        if flag == 0:
            text.append(line.strip())
        else:
            summary.append(line.strip())
    return text,summary
print(clean(l)[0])
print(clean(l)[1])

["Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill.", 'An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.', 'MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court.', 'Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely

In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [7]:
text,summary = clean(l)
text_embeddings = model.encode(text)
summary_embeddings = model.encode(summary)

In [47]:
def k_centers_prob(V, n):
    n1 = n
    centers = []
    cities = list(range(len(V)))
    centers.append(0)
    cities.remove(0)
    while n!= 0:
        city_dict = {}
        for cty in cities:
            min_dist = float("inf")
            for c in centers:
                min_dist = min(min_dist,np.linalg.norm(V[cty] - V[c]))
            city_dict[cty] = min_dist
        new_center = max(city_dict, key = lambda i: city_dict[i])
        centers.append(new_center)
        cities.remove(new_center)
        if n == n1:
            centers.remove(0)
            cities.append(0)
        n = n-1
    return centers

In [48]:
# kmeans = KMeans(n_clusters=len(summary_embeddings)).fit(text_embeddings)

In [49]:
# kmeans.labels_
# min_dist = [float('inf')] * len(summary_embeddings)
# min_node = [-1] * len(summary_embeddings)

# for i in range(len(text_embeddings)):
#     cls = kmeans.labels_[i]
#     dist = np.linalg.norm(text_embeddings[i] - kmeans.cluster_centers_[cls])
#     if dist < min_dist[cls]:
#         min_node[cls] = i
#         min_dist[cls] = dist

In [50]:
# gen_summary = [text[x] for x in min_node]
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'], use_stemmer=True)
# scores2 = scorer.score(" ".join(summary)," ".join(gen_summary))

In [51]:
gen_sum = k_centers_prob(text_embeddings,len(summary_embeddings))

In [52]:
gen_summary = [text[x] for x in gen_sum]

In [53]:
gen_summary

['But when TWC pulled it out of awards contention and shifted its release to March 14, 2014, Dahan lashed out at TWC\'s Harvey Weinstein, over the movie\'s final cut, which the director was in the process of completing. "There are two versions of the film for now, mine and his," Dahan complained, continuing, "They want a commercial film smelling of daisies, taking out anything that exceeds that which is too abrupt, anything that makes it cinematic and breathe with life."',
 'Tim Roth costars as Kelly\'s husband Prince Rainier III, Frank Langella as Kelly\'s priest and confidante, Parker Posey as Grace\'s aid, and Paz Vega ("Spanglish," "Sex and Lucia") as opera singer Maria Callas.',
 "THR: The Weinstein Co. nearing deal to keep 'Grace of Monaco'"]

In [54]:
summary

['The film will premiere on Memorial Day',
 "It opened last year's Cannes Film Festival",
 'A planned March theater release was scrubbed']

In [55]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'], use_stemmer=True)
scores = scorer.score(" ".join(summary)," ".join(gen_summary))

In [56]:
scores

{'rouge1': Score(precision=0.06976744186046512, recall=0.4090909090909091, fmeasure=0.11920529801324503),
 'rouge2': Score(precision=0.0078125, recall=0.047619047619047616, fmeasure=0.013422818791946308),
 'rougeL': Score(precision=0.031007751937984496, recall=0.18181818181818182, fmeasure=0.052980132450331126)}

In [57]:
x = url_hashes_train[20]
l = open(os.path.join("CNN-DM","raw","cnn","stories",x+".story")).readlines()
text,summary = clean(l)
text_embeddings = model.encode(text)
summary_embeddings = model.encode(summary)
gen_sum = k_centers_prob(text_embeddings,len(summary_embeddings))
gen_summary = [text[x] for x in gen_sum]
print(summary)
print("___")
print(gen_summary)
scores = scorer.score(" ".join(summary)," ".join(gen_summary))
scores

['Because of troubles, Lindsay Lohan likely difficult to insure', 'Lohan supposed to appear in Shirley MacLaine film "Poor Things"', "Insurance rates may cost 1 to 3 percent of film's production budget"]
___
['Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed.', "Insurance rates for errant actors can range anywhere from 1 percent to 3 percent of a movie's production budget, which can range from $5 million to $100 million or more, he said.", 'A statement was expected early next week on whether the movie, already delayed this spring due to an earlier rehab stint by Lohan, will proceed with or without her.']


{'rouge1': Score(precision=0.1794871794871795, recall=0.4375, fmeasure=0.2545454545454545),
 'rouge2': Score(precision=0.07792207792207792, recall=0.1935483870967742, fmeasure=0.11111111111111112),
 'rougeL': Score(precision=0.1282051282051282, recall=0.3125, fmeasure=0.1818181818181818)}

In [58]:
x = url_hashes_train[40]
l = open(os.path.join("CNN-DM","raw","cnn","stories",x+".story")).readlines()
text,summary = clean(l)
text_embeddings = model.encode(text)
summary_embeddings = model.encode(summary)
gen_sum = k_centers_prob(text_embeddings,len(summary_embeddings))
gen_summary = [text[x] for x in gen_sum]
print(summary)
print("___")
print(gen_summary)
scores = scorer.score(" ".join(summary)," ".join(gen_summary))
scores

['NEW: Chadian president wants journalists, flight crew released', 'Red Cross, UNICEF, UNHCR interview children that charity tried to fly out of Chad', 'Most are not from Sudan and have families, agencies say', "Six members of Zoe's Ark, 11 others under arrest in Chad"]
___
['Over the weekend, the captain appealed urgently to be rescued, fearing the crew could be harmed or killed, Cajal said.', "The agencies said most of the children also probably come from Chadian villages along Chad's border with Sudan.", "The organization said photographers Marc Garmirian of the Capa news agency and Jean-Daniel Guillou of the Synchro X agency were on assignment for their news organizations and were not part of the charity's efforts.", 'The transfer will take place Monday, according to media reports.']


{'rouge1': Score(precision=0.17647058823529413, recall=0.3409090909090909, fmeasure=0.23255813953488372),
 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0),
 'rougeL': Score(precision=0.09411764705882353, recall=0.18181818181818182, fmeasure=0.12403100775193798)}

In [59]:
%%time

r1_p = []
r1_r = []
r1_f = []

r2_p = []
r2_r = []
r2_f = []

rl_p = []
rl_r = []
rl_f = []

for i in range(len(url_hashes_test[:100])):
    if i>0 and i%10 == 0:
        print(i,"Done")
    x = url_hashes_test[i]
    if os.path.exists(os.path.join("CNN-DM","raw","cnn","stories",x+".story")):
        l = open(os.path.join("CNN-DM","raw","cnn","stories",x+".story")).readlines()
    else:
        l = open(os.path.join("CNN-DM","raw","dm","stories",x+".story")).readlines()
    text,summary = clean(l)
    text_embeddings = model.encode(text)
    summary_embeddings = model.encode(summary)
    gen_sum = k_centers_prob(text_embeddings,len(summary_embeddings))
    gen_summary = [text[x] for x in gen_sum]
#     print(summary)
#     print("___")
#     print(gen_summary)
    scores = scorer.score(" ".join(summary)," ".join(gen_summary))
    r1_p.append(scores['rouge1'].precision)
    r1_r.append(scores['rouge1'].recall)
    r1_f.append(scores['rouge1'].fmeasure)
    
    r2_p.append(scores['rouge2'].precision)
    r2_r.append(scores['rouge2'].recall)
    r2_f.append(scores['rouge2'].fmeasure)
    
    rl_p.append(scores['rougeL'].precision)
    rl_r.append(scores['rougeL'].recall)
    rl_f.append(scores['rougeL'].fmeasure)

10 Done
20 Done
30 Done
40 Done
50 Done
60 Done
70 Done
80 Done
90 Done
CPU times: user 1min 2s, sys: 5.01 s, total: 1min 7s
Wall time: 1min 7s


In [60]:
print("Rouge 1 - Precision : ",np.round(np.mean(np.asarray(r1_p))*100,2))
print("Rouge 1 - Recall    : ",np.round(np.mean(np.asarray(r1_r))*100,2))
print("Rouge 1 - F Measure : ",np.round(np.mean(np.asarray(r1_f))*100,2))
print("---")
print("Rouge 2 - Precision : ",np.round(np.mean(np.asarray(r2_p))*100,2))
print("Rouge 2 - Recall    : ",np.round(np.mean(np.asarray(r2_r))*100,2))
print("Rouge 2 - F Measure : ",np.round(np.mean(np.asarray(r2_f))*100,2))
print("---")
print("Rouge L - Precision : ",np.round(np.mean(np.asarray(rl_p))*100,2))
print("Rouge L - Recall    : ",np.round(np.mean(np.asarray(rl_r))*100,2))
print("Rouge L - F Measure : ",np.round(np.mean(np.asarray(rl_f))*100,2))

Rouge 1 - Precision :  17.83
Rouge 1 - Recall    :  32.81
Rouge 1 - F Measure :  22.02
---
Rouge 2 - Precision :  4.11
Rouge 2 - Recall    :  7.51
Rouge 2 - F Measure :  5.07
---
Rouge L - Precision :  11.45
Rouge L - Recall    :  20.79
Rouge L - F Measure :  14.04


In [61]:
len(url_hashes_test)/100*63/3600

2.0107500000000003

In [63]:
%%time

r1_p = []
r1_r = []
r1_f = []

r2_p = []
r2_r = []
r2_f = []

rl_p = []
rl_r = []
rl_f = []

for i in range(len(url_hashes_test[:100])):
    if i>0 and i%10 == 0:
        print(i,"Done")
    x = url_hashes_test[i]
    if os.path.exists(os.path.join("CNN-DM","raw","cnn","stories",x+".story")):
        l = open(os.path.join("CNN-DM","raw","cnn","stories",x+".story")).readlines()
    else:
        l = open(os.path.join("CNN-DM","raw","dm","stories",x+".story")).readlines()
    text,summary = clean(l)
#     text_embeddings = model.encode(text)
#     summary_embeddings = model.encode(summary)
#     gen_sum = k_centers_prob(text_embeddings,len(summary_embeddings))
    gen_summary = text[:len(summary)]
#     print(summary)
#     print("___")
#     print(gen_summary)
    scores = scorer.score(" ".join(summary)," ".join(gen_summary))
    r1_p.append(scores['rouge1'].precision)
    r1_r.append(scores['rouge1'].recall)
    r1_f.append(scores['rouge1'].fmeasure)
    
    r2_p.append(scores['rouge2'].precision)
    r2_r.append(scores['rouge2'].recall)
    r2_f.append(scores['rouge2'].fmeasure)
    
    rl_p.append(scores['rougeL'].precision)
    rl_r.append(scores['rougeL'].recall)
    rl_f.append(scores['rougeL'].fmeasure)

10 Done
20 Done
30 Done
40 Done
50 Done
60 Done
70 Done
80 Done
90 Done
CPU times: user 415 ms, sys: 21 ms, total: 436 ms
Wall time: 460 ms


In [64]:
print("Rouge 1 - Precision : ",np.round(np.mean(np.asarray(r1_p))*100,2))
print("Rouge 1 - Recall    : ",np.round(np.mean(np.asarray(r1_r))*100,2))
print("Rouge 1 - F Measure : ",np.round(np.mean(np.asarray(r1_f))*100,2))
print("---")
print("Rouge 2 - Precision : ",np.round(np.mean(np.asarray(r2_p))*100,2))
print("Rouge 2 - Recall    : ",np.round(np.mean(np.asarray(r2_r))*100,2))
print("Rouge 2 - F Measure : ",np.round(np.mean(np.asarray(r2_f))*100,2))
print("---")
print("Rouge L - Precision : ",np.round(np.mean(np.asarray(rl_p))*100,2))
print("Rouge L - Recall    : ",np.round(np.mean(np.asarray(rl_r))*100,2))
print("Rouge L - F Measure : ",np.round(np.mean(np.asarray(rl_f))*100,2))

Rouge 1 - Precision :  23.59
Rouge 1 - Recall    :  50.92
Rouge 1 - F Measure :  31.55
---
Rouge 2 - Precision :  9.64
Rouge 2 - Recall    :  21.15
Rouge 2 - F Measure :  12.97
---
Rouge L - Precision :  16.37
Rouge L - Recall    :  35.19
Rouge L - F Measure :  21.85


class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, url_file, raw_path,
                 batch_size,
                 input_size=(384),
                 shuffle=True):
        
        url_list = read_text_file(url_file)
        url_hashes = get_url_hashes(url_list)
        
        self.files = []
        
        for filename in url_hashes:
            cnnpath = os.path.join(raw_path,"cnn","stories",filename + ".story")
            dmpath = os.path.join(raw_path,"dm","stories",filename + ".story")
            if os.path.exists(cnnpath):
                self.files.append(cnnpath)
            elif os.path.exists(dmpath):
                self.files.append(dmpath)
        
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        
        self.n = len(self.files)
    
    def on_epoch_end(self):
        pass
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        path_batch = batches[self.X_col['path']]
        bbox_batch = batches[self.X_col['bbox']]

        name_batch = batches[self.y_col['name']]
        type_batch = batches[self.y_col['type']]

        X_batch = np.asarray([self.__get_input(x, y, self.input_size) for x, y in zip(path_batch, bbox_batch)])

        y0_batch = np.asarray([self.__get_output(y, self.n_name) for y in name_batch])
        y1_batch = np.asarray([self.__get_output(y, self.n_type) for y in type_batch])

        return X_batch, tuple([y0_batch, y1_batch])

    def __getitem__(self, index):

        batches = self.files[index * self.batch_size:min((index + 1) * self.batch_size,self.n-1)]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [ ]:
# max_len = 384
configuration = BertConfig()

In [ ]:
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
save_path = "bert_base_uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)

In [ ]:
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=False)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
# tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
tokenizer2 = BertTokenizer(vocab_file)
# tokenized_sequence = tokenizer2.encode(sequence)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:        
        tokens = tokenizer2.encode(text)
        tokens += [0] * (max_len - len(tokens))
        all_tokens.append(tokens)
    return np.array(all_tokens)

In [ ]:
text,summ = clean(l)
code = bert_encode(text,tokenizer,max_len=256)

In [ ]:
tokenizer.decode([2019])
ans = []
for i in range(len(code[0])):
    x = code[0][i]
    if x != 0:
        ans.append(str(x))
#         print(i,x,tokenizer.decode([x]))
print("--")
print(len(l[0].split()))
print(len(l[0]))
print(len(code[0]))
print(len("".join(ans)))

In [ ]:
code[0]

In [ ]:
import bert

model_dir = "uncased_L-12_H-768_A-12"

bert_params = bert.params_from_pretrained_ckpt(model_dir)
l_bert = bert.BertModelLayer.from_params(bert_params, name="bert")

In [ ]:
from tensorflow import keras

max_seq_len = 128
l_input_ids      = keras.layers.Input(shape=(max_seq_len,), dtype='int32')
l_token_type_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32')

# using the default token_type/segment id 0
output = l_bert(l_input_ids)                              # output: [batch_size, max_seq_len, hidden_size]
model = keras.Model(inputs=l_input_ids, outputs=output)
model.build(input_shape=(None, max_seq_len))

In [ ]:
bert_ckpt_file   = os.path.join(model_dir, "bert_model.ckpt")
bert.load_stock_weights(l_bert, bert_ckpt_file)

In [ ]:
model.p